In [49]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx

import matplotlib.pyplot as plt

sys.path.append(r'../../GOSTNets/GOSTNets')
import GOSTnet as gn

from shapely.geometry import Point, LineString

In [27]:
scenario_var = "_Feb_v5"
cost_attr = 'cost'
res_folder = r"./"
G_pickle  = os.path.join(res_folder, "asia_railways_ver3.pickle")

# Input files
edges_geography = r"./geographic_data/edges.shp"
edges_data = gpd.read_file(edges_geography)

# Output files
complete_volume_map = os.path.join(res_folder, "complete_equal_volume_map_Feb_v5.csv")
trade_volume_map = os.path.join(res_folder, "trade_volume_map_Feb_v5.csv")

# need this for now...
nodes_geography = r"./geographic_data/rail_nodes_Feb_v5.csv"
nodes_data = pd.read_csv(nodes_geography, dtype={'X': np.float64, 'Y': np.float64})
nodes_data['Point'] = [Point(x, y) for x, y in zip(nodes_data['X'],nodes_data['Y'])]
nodes_data['node_id'] = nodes_data['node_id'].astype(str)

OD_df = pd.read_csv(os.path.join(res_folder, "asia_railways_output_OD_matrix{}.csv".format(scenario_var)), index_col=0)
G = nx.read_gpickle(G_pickle)

trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_Feb_v5.csv"))

In [28]:
len(trade_flows)

100

In [29]:
edges_data

source  target  link_id  length         time    name_x  \
0        38      69        1    1304    28.977778   Kashgar   
1        40      69       29     791    17.577778   Khorgos   
2        38      70        2     174     3.866667   Kashgar   
3        36      70        3     259    10.360000  Kara Suu   
4        36      49      104      22     0.880000  Kara Suu   
..      ...     ...      ...     ...          ...       ...   
104      20      91      114    1560    47.640000      None   
105      12      66       19     623    26.520000      None   
106      45      63       88    1040    44.770000      None   
107      20      92      115    3000  1091.620000      None   
108      81      92      116    3000    91.620000      None   

                                              geometry  
0    LINESTRING (75.99371 39.46769, 89.18467 42.94998)  
1    LINESTRING (80.41190 44.21607, 89.18467 42.94998)  
2    LINESTRING (75.99371 39.46769, 75.41076 40.57480)  
3    LINESTRING (72.86264 40.70247, 75.41076 40.57480)  
4    LINESTRING (72.86264 40.70247, 72.81063 40.52017)  
..                                                 ...  
104  LINESTRING (60.64593 25.29688, 72.92575 19.02477)  
105  LINESTRING (52.23207 30.26452, 59.21328 35.27869)  
106  LINESTRING (59.61583 36.26611, 51.42322 35.69698)  
107  LINESTRING (60.64593 25.29688, 60.76567 14.76131)  
108  LINESTRING (60.76567 14.76131, 62.29135 25.23314)  

[109 rows x 7 columns]

In [30]:
nodes_data[:18]

node_id          name country  border  gauge_change  \
0        1         Aktau     KAZ   False         False   
1        2        Aktobe     KAZ   False         False   
2        3       Aktogal     KAZ   False         False   
3        4        Almaty     KAZ   False         False   
4        5       Andijan     UBZ    True         False   
5        6        Angren     UBZ   False         False   
6        7          Arys     KAZ   False         False   
7        8      Ashgabat     TKM   False         False   
8        9         Atrek     TKM   False         False   
9       10          Baku     AZE    True         False   
10      11       Balkash     KAZ   False         False   
11      12          Barq     IRA   False         False   
12      14       Bereket     TKM   False         False   
13      15        Berlin     GER   False         False   
14      16        Beynau     KAZ   False         False   
15      17        Birlik     KAZ   False         False   
16      18         Brest     BEL    True          True   
17     18b  Brest_border     BEL    True          True   

    if_border_is_an_origin_or_destination      Name          X          Y  \
0                                   False     Aktau  51.194236  43.650086   
1                                   False    Aktobe  57.166930  50.283883   
2                                   False   Aktogal  79.922600  46.851951   
3                                   False    Almaty  76.857735  43.229664   
4                                   False   Andijan  72.284590  40.814750   
5                                   False    Angren  70.074230  41.008453   
6                                   False      Arys  68.813051  42.433659   
7                                   False  Ashgabat  58.316298  37.943921   
8                                   False     Atrek  54.021612  37.414975   
9                                    True      Baku  49.866373  40.409438   
10                                  False   Balkash  74.974699  46.840817   
11                                  False      Barq  52.232000  30.264000   
12                                  False   Bereket  55.519378  39.246962   
13                                  False    Berlin  13.419359  52.522534   
14                                  False    Beynau  55.181848  45.322236   
15                                  False    Birlik  73.916840  43.652882   
16                                   True     Brest  23.763666  52.099651   
17                                   True     Brest  23.763666  52.099651   

                                    Point  
0          POINT (51.19423615 43.6500856)  
1         POINT (57.16692971 50.28388335)  
2    POINT (79.92260014999999 46.8519507)  
3         POINT (76.85773523 43.22966422)  
4    POINT (72.28458996000001 40.8147501)  
5         POINT (70.07422952 41.00845288)  
6          POINT (68.8130505 42.43365853)  
7         POINT (58.31629783 37.94392081)  
8         POINT (54.02161162 37.41497489)  
9         POINT (49.86637319 40.40943762)  
10        POINT (74.97469911 46.84081729)  
11                  POINT (52.232 30.264)  
12        POINT (55.51937803 39.24696232)  
13        POINT (13.41935921 52.52253403)  
14        POINT (55.18184777 45.32223604)  
15        POINT (73.91684012 43.65288192)  
16  POINT (23.76366624 52.09965070000001)  
17  POINT (23.76366624 52.09965070000001)

In [31]:
OD_df

38          69          70          36           5          25  \
38     0.171170   42.271170   17.641170   69.156170  107.971170  122.511170   
69    42.271170    0.171170   59.741170  111.256170  130.181170  115.641170   
70    17.641170   59.741170    0.171170   51.686170   90.501170  105.041170   
36    69.156170  111.256170   51.686170    0.171170   38.986170   53.526170   
5    107.971170  130.181170   90.501170   38.986170    0.171170   14.711170   
..          ...         ...         ...         ...         ...         ...   
90   429.221170  402.531170  411.751170  360.236170  321.421170  306.881170   
91   475.391171  448.701171  457.921171  406.406171  367.591171  353.051171   
92   541.021170  514.331170  523.551170  472.036170  433.221170  418.681170   
18b  340.711170  298.611170  358.181170  322.656170  283.841170  269.301170   
68b  183.231171  156.541171  165.761171  114.246171   75.431171   60.891171   

             35          54          37          65  ...          80  \
38   137.791170  141.701170  148.141170  160.201170  ...  334.591170   
69   118.921170  115.011170  121.451170  133.511170  ...  307.901170   
70   120.321170  124.231170  130.671170  142.731170  ...  317.121170   
36    68.806170   72.716170   79.156170   91.216170  ...  265.606170   
5     29.991170   33.901170   40.341170   52.401170  ...  226.791170   
..          ...         ...         ...         ...  ...         ...   
90   291.601170  287.691170  281.251170  269.191170  ...  125.821170   
91   337.771171  333.861171  330.961171  343.021171  ...  346.151170   
92   403.401170  399.491170  393.051170  380.991170  ...  206.601170   
18b  272.581170  268.671170  275.111170  287.171170  ...  461.561170   
68b   45.611171   41.701171   38.801171   50.861171  ...  225.251171   

             81          89          93          94          90          91  \
38   399.401170  249.111170  304.451170  334.331170  429.221170  475.391171   
69   372.711170  222.421170  277.761170  307.641170  402.531170  448.701171   
70   381.931170  231.641170  286.981170  316.861170  411.751170  457.921171   
36   330.416170  180.126170  235.466170  265.346170  360.236170  406.406171   
5    291.601170  141.311170  196.651170  226.531170  321.421170  367.591171   
..          ...         ...         ...         ...         ...         ...   
90    61.011170  180.281170  124.941170   95.061170    0.171170  471.801170   
91   410.961170  365.281170  420.621170  450.501170  471.801170    0.171170   
92   141.791170  351.961170  327.401170  297.521170  202.631170  552.581170   
18b  526.371170  376.081170  431.421170  461.301170  556.191170  574.071171   
68b  290.061171  139.771171  195.111171  224.991171  319.881171  292.331170   

             92         18b         68b  
38   541.021170  340.711170  183.231171  
69   514.331170  298.611170  156.541171  
70   523.551170  358.181170  165.761171  
36   472.036170  322.656170  114.246171  
5    433.221170  283.841170   75.431171  
..          ...         ...         ...  
90   202.631170  556.191170  319.881171  
91   552.581170  574.071171  292.331170  
92     0.171170  667.991170  431.681171  
18b  667.991170    0.171170  310.201171  
68b  431.681171  310.201171    0.171170  

[90 rows x 90 columns]

# Map network volume based on ALL least-cost trips

In [32]:
def tabulate_edges(route):
    edge_table = []
    for idx in range(0, len(route) - 1):
        edge_table.append([route[idx], route[idx+1]])
    return(edge_table)

"""
def get_route(route, nodes_data):
    all_nodes = []
    for x in route:
        try:
            cur_node = nodes_data.loc[nodes_data['node_id'] == x]
            all_nodes.append(Point(cur_node.X, cur_node.Y))
        except:
            pass
    if len(all_nodes) > 1:
        return(LineString(all_nodes))
    else:
        return(None)
"""

"\ndef get_route(route, nodes_data):\n    all_nodes = []\n    for x in route:\n        try:\n            cur_node = nodes_data.loc[nodes_data['node_id'] == x]\n            all_nodes.append(Point(cur_node.X, cur_node.Y))\n        except:\n            pass\n    if len(all_nodes) > 1:\n        return(LineString(all_nodes))\n    else:\n        return(None)\n"

In [33]:
all_res = []
complete_edges = []

count = 0

for origin, row in OD_df.iterrows():    
    for destination, value in row.items():
        try:
            origin = int(origin)
        except:
            pass
        try:
            destination = int(destination)
        except:
            pass
        
        count = count + 1
        route = nx.bidirectional_shortest_path(G, origin, destination)
        # print(route)
        edge_table = tabulate_edges(route)
        print(edge_table)
        complete_edges = complete_edges + edge_table 
        #all_res.append([origin, destination, get_route(route, nodes_data)])


[]
[[38, 69]]
[[38, 70]]
[[38, 70], [70, 36]]
[[38, 70], [70, 36], [36, 5]]
[[38, 70], [70, 36], [36, 5], [5, 25]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37], [37, 65]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37], [37, 65], [65, 31]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55]]
[[38, 70], [70, 36], [36, 5], [5, 25], [25, 35]

[[68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56]]
[[68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48]]
[[68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87]]
[[68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 2]]
[[68, '68b'], ['68b', 43], [43, 8], [8, 67], [67, 10], [10, 62], [62, 82], [82, 34], [34, 73], [73, 18], [18, 75]]
[[68, '68b'], ['68b', 43], [43, 8], [8, 67], [67, 10], [10, 62], [62, 82], [82, 34], [34, 32], [32, 15], [15, 88]]
[[68, '68b'], ['68b', 43], [43, 8], [8, 67], [67, 10], [10, 62], [62, 82], [82, 34], [34, 32], [32, 15]]
[[68, 39], [39, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 21]]
[[68, 39], [39, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 21], [21, 3]]
[[68, 39], [39, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 21], [21, 3], [3, 11]]
[[68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 78], [78, 77], [77, 46]]
[[68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 78], [78, 77]]


[[42, 59], [59, 33], [33, 83]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 40]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17], [17, 4]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86], [86, 30]]
[[42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14]]
[[42, 59]

[[17, 7], [7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 32]]
[[17, 7], [7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 73], [73, 18]]
[[17, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74], [74, 27]]
[[17, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74]]
[[17, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29]]
[[17, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56]]
[[17, 7], [7, 53], [53, 2], [2, 87], [87, 48]]
[[17, 7], [7, 53], [53, 2], [2, 87]]
[[17, 7], [7, 53], [53, 2]]
[[17, 7], [7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 73], [73, 18], [18, 75]]
[[17, 7], [7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 32], [32, 15], [15, 88]]
[[17, 7], [7, 53], [53, 16], [16, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 32], [32, 15]]
[[17, 4], [4, 21]]
[[17, 4], [4, 21], [21, 3]]
[[17, 4], [4, 21], [21, 3], [3, 11]]
[[17, 4], [4, 21], [2

[[86, 30], [30, 14], [14, 9], [9, 85], [85, 28], [28, 57], [57, 26], [26, 45], [45, 66], [66, 12], [12, 76], [76, 20], [20, 91]]
[[86, 30], [30, 14], [14, 9], [9, 85], [85, 28], [28, 57], [57, 26], [26, 45], [45, 66], [66, 12], [12, 76], [76, 20], [20, 92]]
[[86, 71], [71, 1], [1, 10], [10, 62], [62, 82], [82, 34], [34, 73], [73, 18], [18, '18b']]
[[86, 71], [71, 1], [1, 10], [10, 67], [67, 8], [8, 43], [43, '68b']]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 17], [17, 4], [4, 40], [40, 69], [69, 38]]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 17], [17, 4], [4, 40], [40, 69]]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 17], [17, 4], [4, 40], [40, 69], [69, 38], [38, 70]]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 60], [60, 54], [54, 35], [35, 25], [25, 5], [5, 36]]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 7], [7, 60], [60, 54], [54, 35], [35, 25], [25, 5]]
[[30, 86], [86, 71], [71, 1], [1, 16], [16, 

...

[[49, 36], [36, 70], [70, 38]]
[[49, 36], [36, 70], [70, 38], [38, 69]]
[[49, 36], [36, 70]]
[[49, 36]]
[[49, 36], [36, 5]]
[[49, 36], [36, 5], [5, 25]]
[[49, 36], [36, 5], [5, 25], [25, 35]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37], [37, 65]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 37], [37, 65], [65, 31]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55]]
[[49, 36], [36, 5], [5, 25], [25, 35], [35, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58,

[[80, 79], [79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53]]
[[80, 79], [79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16]]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, 8], [8, 67], [67, 10], [10, 1]]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, 8], [8, 67], [67, 10], [10, 1], [1, 71]]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14], [14, 30], [30, 86]]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14], [14, 30]]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14]]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9]]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57],

[[93, 89], [89, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32], [32, 15]]
[[93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 21]]
[[93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 21], [21, 3]]
[[93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 21], [21, 3], [3, 11]]
[[93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 78], [78, 77], [77, 46]]
[[93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 78], [78, 77]]
[[93, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 78]]
[[93, 89], [89, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52]]
[[93, 89], [89, 42], [42, 59], [59, 33], [33, 83], [83, 66], [66, 45], [45, 63], [63, 52], [5

[[91, 20], [20, 92], [92, 81], [81, 90]]
[]
[[91, 20], [20, 92]]
[[91, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 73], [73, 18], [18, '18b']]
[[91, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b']]
[[92, 81], [81, 80], [80, 79], [79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 35], [35, 25], [25, 5], [5, 36], [36, 70], [70, 38]]
[[92, 81], [81, 80], [80, 79], [79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 40], [40, 69]]
[[92, 81], [81, 80], [80, 79], [79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 35], [35, 25], [25, 5], [5, 36], [36, 70]]
[[92, 81], [81, 80], [80, 79], [79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 35], [35, 25], [25, 5], [5, 36]]
[[92, 81], [81, 80], [80, 79], [79, 33], [33, 59], [59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [5

[['68b', 43], [43, 58], [58, 55], [55, 45], [45, 66], [66, 83], [83, 33], [33, 79], [79, 80]]
[['68b', 43], [43, 58], [58, 55], [55, 45], [45, 66], [66, 83], [83, 33], [33, 79], [79, 80], [80, 81]]
[['68b', 68], [68, 39], [39, 37], [37, 65], [65, 31], [31, 42], [42, 89]]
[['68b', 68], [68, 39], [39, 37], [37, 65], [65, 31], [31, 42], [42, 89], [89, 93]]
[['68b', 68], [68, 39], [39, 37], [37, 65], [65, 31], [31, 42], [42, 89], [89, 93], [93, 94]]
[['68b', 68], [68, 39], [39, 37], [37, 65], [65, 31], [31, 42], [42, 89], [89, 93], [93, 94], [94, 90]]
[['68b', 43], [43, 58], [58, 55], [55, 45], [45, 66], [66, 12], [12, 76], [76, 20], [20, 91]]
[['68b', 43], [43, 58], [58, 55], [55, 45], [45, 66], [66, 12], [12, 76], [76, 20], [20, 92]]
[['68b', 43], [43, 8], [8, 67], [67, 10], [10, 62], [62, 82], [82, 34], [34, 73], [73, 18], [18, '18b']]
[]


In [34]:
count

8100

In [35]:
len(complete_edges)

66026

In [36]:
complete_edges[6]

[36, 5]

In [37]:
complete = pd.DataFrame(complete_edges, columns=['o', 'd'])
complete['w'] = 1
complete_count = pd.DataFrame(complete.groupby(['o','d']).count())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [38]:
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='d', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]

In [39]:
complete_count

o   d    w                                           geometry
0     1  10  830  LINESTRING (51.19423615 43.6500856, 49.8663731...
1    62  10  744  LINESTRING (44.82710578 41.71513710000001, 49....
2    67  10  382  LINESTRING (52.96318620000001 40.02016264, 49....
3     1  16  942  LINESTRING (51.19423615 43.6500856, 55.1818477...
4    53  16  946  LINESTRING (60.31394877 47.04978092, 55.181847...
..   ..  ..  ...                                                ...
203  94  90   63  LINESTRING (71.56378993 34.10942691, 67.001097...
204  89  93  207  LINESTRING (69.19380137 34.50670823, 71.111711...
205  94  93  134  LINESTRING (71.56378993 34.10942691, 71.111711...
206  90  94   63  LINESTRING (67.00109753 24.89698664, 71.563789...
207  93  94  134  LINESTRING (71.11171118999999 34.19162304, 71....

[208 rows x 4 columns]

In [40]:
complete_count.to_csv(complete_volume_map)

# Map specific routes with trade flows

In [41]:
#trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_v1.csv"))
complete_edges = []
for idx, row in trade_flows.iterrows():    
    
    try:
        origin = row['Node number']
        destination = row['Node Number']
        route = nx.bidirectional_shortest_path(G, origin, destination)
        edge_table = tabulate_edges(route)
        [x.append(int(row['Trade volume'].replace(",", ""))) for x in edge_table]
        complete_edges = complete_edges + edge_table 
    except:
        pass
        

    


In [42]:
len(complete_edges)

385

In [43]:
complete_edges

[[36, 5, 18214],
 [5, 25, 18214],
 [25, 35, 18214],
 [35, 54, 18214],
 [54, 37, 18214],
 [37, 65, 18214],
 [65, 31, 18214],
 [31, 42, 18214],
 [42, 89, 18214],
 [36, 5, 373],
 [5, 25, 373],
 [25, 35, 373],
 [35, 54, 373],
 [54, 37, 373],
 [37, 65, 373],
 [65, 31, 373],
 [31, 42, 373],
 [42, 89, 373],
 [89, 93, 373],
 [93, 94, 373],
 [94, 90, 373],
 [36, 70, 6957371],
 [70, 38, 6957371],
 [38, 69, 6957371],
 [69, 40, 6957371],
 [40, 4, 6957371],
 [36, 5, 1737602],
 [5, 25, 1737602],
 [25, 35, 1737602],
 [35, 54, 1737602],
 [54, 37, 1737602],
 [37, 22, 1737602],
 [36, 5, 1218590],
 [5, 25, 1218590],
 [36, 5, 1218590],
 [5, 25, 1218590],
 [25, 35, 1218590],
 [35, 54, 1218590],
 [54, 60, 1218590],
 [36, 5, 1218590],
 [5, 25, 1218590],
 [25, 35, 1218590],
 [35, 54, 1218590],
 [91, 20, 1078810],
 [20, 92, 1078810],
 [92, 81, 1078810],
 [81, 90, 1078810],
 [90, 94, 1078810],
 [94, 93, 1078810],
 [93, 89, 1078810],
 [91, 20, 687606],
 [20, 76, 687606],
 [76, 12, 687606],
 [12, 66, 687606],
 [6

In [44]:
complete = pd.DataFrame(complete_edges, columns=['o', 'd', 'w'])
complete_count = pd.DataFrame(complete.groupby(['o','d']).sum())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [45]:
complete_count

o   d         w
0     1  10      9906
1     4  17  26557061
2     5  25   5800427
3     5  36   1108161
4     7  17   5474893
..  ...  ..       ...
75   93  94       373
76   94  90       373
77   94  93  22260730
78  68b  43   1022230
79  68b  68   5670202

[80 rows x 3 columns]

In [46]:
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='d', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]

In [47]:
complete_count

o   d         w                                           geometry
0    1  10      9906  LINESTRING (51.19423615 43.6500856, 49.8663731...
1    4  17  26557061  LINESTRING (76.85773523 43.22966422, 73.916840...
2    7  17   5474893  LINESTRING (68.8130505 42.43365853, 73.9168401...
3    5  25   5800427  LINESTRING (72.28458996000001 40.8147501, 71.7...
4   35  25   6163925  LINESTRING (67.85315559 40.13305512, 71.797853...
..  ..  ..       ...                                                ...
75  89  93       373  LINESTRING (69.19380137 34.50670823, 71.111711...
76  94  93  22260730  LINESTRING (71.56378993 34.10942691, 71.111711...
77  90  94  22260730  LINESTRING (67.00109753 24.89698664, 71.563789...
78  93  94       373  LINESTRING (71.11171118999999 34.19162304, 71....
79  91  20   5811584  LINESTRING (72.92574618 19.02476851, 60.645928...

[80 rows x 4 columns]

In [48]:
complete_count.to_csv(trade_volume_map)